In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn import linear_model
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [2]:
df = pd.read_csv('NYState2013orig.csv',skiprows=4)
df.head()

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,Adams Village,"1,861",0,0.000,nan,0,0,0,12,2,10,0,0.000
1,Addison Town and Village,"2,577",3,0.000,nan,0,0,3,24,3,20,1,0.000
2,Akron Village,"2,846",3,0.000,nan,0,0,3,16,1,15,0,0.000
3,Albany,"97,956",791,8.000,nan,30,227,526,"4,090",705,"3,243",142,nan
4,Albion Village,"6,388",23,0.000,nan,3,4,16,223,53,165,5,nan


In [3]:
# Just a few in the Population column (e.g. row 278) are not actually strings. Convert them all to strings first.
df.Population=df.Population.apply(lambda x:str(x))

In [4]:
# Take out commas in Population column
df.Population = df.Population.str.replace(',','')

In [5]:
# Remove the bottom three rows of the dataframe that are footnotes
df=df.iloc[0:-3,:]

In [6]:
# Make Population an integer
df.Population=df.Population.apply(lambda x:int(x))

In [7]:
# Create a Population squared column
df['Population_sq']=df.Population**2

In [8]:
df.columns = [c.replace(' ', '_') for c in df.columns]

In [9]:
df.columns = [c.replace('\n', '_') for c in df.columns]

In [10]:
df.Property_crime = df.Property_crime.str.replace(',','')

In [11]:
df.Property_crime = df.Property_crime.apply(lambda x:int(x))

In [12]:
# Create Property Crime per capita column
df['prop_crime_per_capita']=df.Property_crime/df.Population

In [13]:
# Take out commas in Robbery column
df.Robbery = df.Robbery.str.replace(',','')

In [14]:
# Make Robbery an integer
df.Robbery=df.Robbery.apply(lambda x:int(x))

In [15]:
# Before making Robbery categorical, make a copy of the column for later:
df['RobberyNum']=df['Robbery']

In [16]:
for x in range (len(df)):
    if df.loc[df.index[x],'Robbery'] > 0:
        df.loc[df.index[x],'Robbery'] = 1

In [17]:
df.Robbery.head()

0    0
1    0
2    0
3    1
4    1
Name: Robbery, dtype: int64

In [18]:
def convert_value(num):
    if num > 0:
        return 1
    else: 
        return 0

In [19]:
# Make Robbery categorical
df.Robbery=df.Robbery.apply(lambda x:convert_value(x))

In [20]:
df.Robbery.head()

0    0
1    0
2    0
3    1
4    1
Name: Robbery, dtype: int64

In [21]:
# Re-name the crazy "Murder and nonnegligent manslaughter" column
df.columns = [c.replace(' ', '_') for c in df.columns]

In [22]:
df.columns

Index(['City', 'Population', 'Violent_crime',
       'Murder_and_nonnegligent_manslaughter', 'Rape_(revised_definition)1',
       'Rape_(legacy_definition)2', 'Robbery', 'Aggravated_assault',
       'Property_crime', 'Burglary', 'Larceny-_theft', 'Motor_vehicle_theft',
       'Arson3', 'Population_sq', 'prop_crime_per_capita', 'RobberyNum'],
      dtype='object')

In [23]:
df.columns = [c.replace('\n', '_') for c in df.columns]

In [24]:
df.columns

Index(['City', 'Population', 'Violent_crime',
       'Murder_and_nonnegligent_manslaughter', 'Rape_(revised_definition)1',
       'Rape_(legacy_definition)2', 'Robbery', 'Aggravated_assault',
       'Property_crime', 'Burglary', 'Larceny-_theft', 'Motor_vehicle_theft',
       'Arson3', 'Population_sq', 'prop_crime_per_capita', 'RobberyNum'],
      dtype='object')

In [25]:
# Make the murder column categorical
df.Murder_and_nonnegligent_manslaughter=df.Murder_and_nonnegligent_manslaughter.apply(lambda x:convert_value(x))

In [26]:
df.head()

,City,Population,Violent_crime,Murder_and_nonnegligent_manslaughter,Rape_(revised_definition)1,Rape_(legacy_definition)2,Robbery,Aggravated_assault,Property_crime,Burglary,Larceny-_theft,Motor_vehicle_theft,Arson3,Population_sq,prop_crime_per_capita,RobberyNum
0,Adams Village,1861,0,0,nan,0,0,0,12,2,10,0,0.000,3463321,0.006,0
1,Addison Town and Village,2577,3,0,nan,0,0,3,24,3,20,1,0.000,6640929,0.009,0
2,Akron Village,2846,3,0,nan,0,0,3,16,1,15,0,0.000,8099716,0.006,0
3,Albany,97956,791,1,nan,30,1,526,4090,705,"3,243",142,nan,9595377936,0.042,227
4,Albion Village,6388,23,0,nan,3,1,16,223,53,165,5,nan,40806544,0.035,4


In [27]:
df_clean=df[['City','Population','Population_sq','Robbery','Murder_and_nonnegligent_manslaughter','Property_crime']]

In [28]:
df_clean.head()

,City,Population,Population_sq,Robbery,Murder_and_nonnegligent_manslaughter,Property_crime
0,Adams Village,1861,3463321,0,0,12
1,Addison Town and Village,2577,6640929,0,0,24
2,Akron Village,2846,8099716,0,0,16
3,Albany,97956,9595377936,1,1,4090
4,Albion Village,6388,40806544,1,0,223


In [29]:
df_features=df[['Population','Population_sq','Robbery','Murder_and_nonnegligent_manslaughter']]

In [30]:
df_target=df['Property_crime']

In [31]:
regr = linear_model.LinearRegression()

# Fit our model to our data.
regr.fit(df_features,df_target)

# Display the attributes we calculated.
print('Coefficients: \n', regr.coef_)
print('Intercept: \n', regr.intercept_)

print('R squared: \n',regr.score(df_features,df_target))


Coefficients: 
 [  3.46570268e-02  -2.11108019e-09  -9.62774363e+01   1.51866535e+01]
Intercept: 
 -109.575335623
R squared: 
 0.996124710499


In [32]:
df_features.head()

,Population,Population_sq,Robbery,Murder_and_nonnegligent_manslaughter
0,1861,3463321,0,0
1,2577,6640929,0,0
2,2846,8099716,0,0
3,97956,9595377936,1,1
4,6388,40806544,1,0


In [33]:
# Try the robbery feature as original number, rather than the categorical version.
df_features=df[['Population','Population_sq','Murder_and_nonnegligent_manslaughter','RobberyNum']]

In [34]:
df_features.head()

,Population,Population_sq,Murder_and_nonnegligent_manslaughter,RobberyNum
0,1861,3463321,0,0
1,2577,6640929,0,0
2,2846,8099716,0,0
3,97956,9595377936,1,227
4,6388,40806544,0,4


In [35]:
regr = linear_model.LinearRegression()

# Fit our model to our data.
regr.fit(df_features,df_target)

# Display the attributes we calculated.
print('Coefficients: \n', regr.coef_)
print('Intercept: \n', regr.intercept_)

print('R squared: \n',regr.score(df_features,df_target))

Coefficients: 
 [  1.40623373e-02  -1.53307561e-09   6.70428655e+01   6.87936698e+00]
Intercept: 
 30.6866936661
R squared: 
 0.998524567525


In [36]:
#Seems better, actually, but population_sq seems to be adding very little. Try without?
df_features=df[['Population','Murder_and_nonnegligent_manslaughter','RobberyNum']]

In [37]:
df_features.head()

,Population,Murder_and_nonnegligent_manslaughter,RobberyNum
0,1861,0,0
1,2577,0,0
2,2846,0,0
3,97956,1,227
4,6388,0,4


In [38]:
regr = linear_model.LinearRegression()

# Fit our model to our data.
regr.fit(df_features,df_target)

# Display the attributes we calculated.
print('Coefficients: \n', regr.coef_)
print('Intercept: \n', regr.intercept_)

print('R squared: \n',regr.score(df_features,df_target))

Coefficients: 
 [ -1.92671540e-03   5.43653696e+02   8.22520859e+00]
Intercept: 
 193.561889846
R squared: 
 0.997132425624


In [39]:
# Maybe we should predict property crime per capita, because the variation in the sheer number of property crimes 
# could be explained by an increase in population alone
df_target=df['prop_crime_per_capita']
df_features=df[['Population','Population_sq','Murder_and_nonnegligent_manslaughter','RobberyNum']]

In [40]:
regr = linear_model.LinearRegression()

# Fit our model to our data.
regr.fit(df_features,df_target)

# Display the attributes we calculated.
print('Coefficients: \n', regr.coef_)
print('Intercept: \n', regr.intercept_)

print('R squared: \n',regr.score(df_features,df_target))

Coefficients: 
 [ -3.31774900e-08  -5.29787278e-15   6.34752530e-03   3.35960989e-05]
Intercept: 
 0.0185447520951
R squared: 
 0.0668868715974


In [41]:
# Evidently that linear regression model does not explain property crimes per capita